In [7]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry

In [8]:
ENDPOINT = "https://westus2.api.cognitive.microsoft.com"

# Replace with a valid key
training_key = "911653b68cfa4975809f02e8658e2987"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

In [9]:
project_id = "610fd33e-e7c8-4429-a3ac-1dbedcbc3a2e"
print(trainer)

In [10]:
# Make two tags in the new project
man_with_hat_tag = trainer.create_tag(project_id, "Man with a hat")

In [12]:
base_image_url = "Images"

print("Adding images...")

image_list = []

for image_num in range(1, 7):
    file_name = "man_with_a_hat{}.jpg".format(image_num)
    with open(base_image_url + "/Humans/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[man_with_hat_tag.id]))

upload_result = trainer.create_images_from_files(project_id, images=image_list)
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


In [19]:
import time

print ("Training...")
iteration = trainer.train_project(project_id)
print(iteration)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project_id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...


CustomVisionErrorException: Nothing changed since last training

In [22]:
publish_iteration_name = "comic_describer"
prediction_resource_id = "/subscriptions/b8c23406-f9b5-4ccb-8a65-a8cb5dcd6a5a/resourceGroups/hichandocustomai/providers/Microsoft.CognitiveServices/accounts/hichandocustomai_prediction"

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project_id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!
